In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Dec 29 17:26:52 2024

@author: user
"""
import time
start = time.time()
# =============================================================================
# 
import os
import netCDF4 as nc
import pandas as pd

import numpy as np
import numpy.ma as ma

import xarray as xr

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.style as mplstyle
import matplotlib.dates as mdates

import cartopy.crs as ccrs
import cartopy.feature as cfeat
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.io.shapereader import Reader
import cartopy.io.shapereader as shpreader
from cartopy.mpl.ticker import LongitudeFormatter,LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import netCDF4 as nc
from netCDF4 import Dataset

import shapefile


import matplotlib.dates as mdates

# =============================================================================



import sys
import os
# 将目标模块所在的路径添加到 sys.path 中
sys.path.append("/home/yfdong/data/work/LF-SAM/code/Library")
from MeteoChartPlot import (plot_AWB_DIFF , plot_AWB_Mean ,plot_TWB_DIFF ,
                            plot_TWB_Mean , preprocess_WB_Df, add_TickGrid)
from PreprocessVar import (
    get_MONTH_abbr, get_pentad, get_letter,
    preprocess_var, read_var,Meiyu_configs,
    Read_VarsFromVarsDict, experiments_types,
    load_geodata, SubsetDomainConfigure ,create_domain_mask,get_letter, UTC_Convert_to_LST
)
import datetime
DataPath = '/raid61/yfdong/data/work/LF-SAM/output'
# Define the subset domain for analysis
SubsetDomain = [28.8, 30.8, 116.85, 120.0]  # [lat_min, lat_max, lon_min, lon_max]
# Area = (SubsetDomain[1] - SubsetDomain[0]) * (SubsetDomain[3] - SubsetDomain[2]) * 100 * 100  # Approximate area in km²
# print(f"Analysis Area: {Area} km²")
# 更准确计算区域面积（单位：km²）
import numpy as np


In [ ]:
# 读取数据
Wdf_total = pd.read_csv('Wdf_total.csv')
Hdf_total = pd.read_csv('Hdf_total.csv')

In [ ]:
# =============================================================================
EventState = 'Dur'
# FeedbackState = 'Positive'
NumericalColumns = [
        'ET', 'HADV', 'VADV','Fin', 'Fout',
       'SouthQin', 'SouthQout', 'NorthQin', 'NorthQout', 'WestQin', 'WestQout',
       'EastQin', 'EastQout',
       'Prec', 
       'PW',  'AWBR', 'Qs', 'Qn', 'Qe', 'Qw', 'ETout',
    #    'ZQ','MQ',
        'PRR', 'Pe', 'Pa', 
    #    'Qa', 'Qe'
        ]
print('=============================================================================')
print(f'Event State: {EventState}')
# print(f'Feedback State: {FeedbackState}')
print('=============================================================================')
# =============================================================================
def extract_event_data(Wdf_total, Hdf_total, FeedbackState):
    import pandas as pd
    Event_df = pd.read_excel(f'/home/yfdong/data/work/LF-SAM/code/AnalysisModule/HuangshanPrecAnalysis/EventDateDf_{FeedbackState}.xlsx')
    Event_subset_df = Event_df.copy()
    # print(Event_subset_df)
    # ----------------------------- 提取强降水事件发生时期的水汽通量输送 --------------------
    from pandas.api.types import is_datetime64_any_dtype as is_datetime

    # 确保时间列正确
    if not is_datetime(Event_subset_df['DurStartDate']):
        Event_subset_df[['DurStartDate','DurEndDate']] = Event_subset_df[['DurStartDate','DurEndDate']].apply(pd.to_datetime)

    # 使用条件合并 (merge) 替代循环
    merged = Wdf_total.assign(key=1).merge(
        Event_subset_df.assign(key=1), 
        on='key',
        suffixes=('', '_event')
    ).drop('key', axis=1)

    # 筛选时间范围内的数据
    mask = (merged['Date_LST'] >= merged['DurStartDate']) & (merged['Date_LST'] <= merged['DurEndDate'])
    Wdf_total_event = merged.loc[mask].reset_index(drop=True)

    # 使用条件合并 (merge) 替代循环
    merged = Hdf_total.assign(key=1).merge(
        Event_subset_df.assign(key=1), 
        on='key',
        suffixes=('', '_event')
    ).drop('key', axis=1)

    # 筛选时间范围内的数据
    mask = (merged['Date_LST'] >= merged['DurStartDate']) & (merged['Date_LST'] <= merged['DurEndDate'])
    Hdf_total_event = merged.loc[mask].reset_index(drop=True)

    # ---------------------- 计算WRF-H和WRF-S之间的模拟差异 ---------------------
    # NumericalColumns = ['PW','Q', 'ZQ', 'Prec', 'ET',
    #                 #     'TRF','TSS','SFROFF','SBROFF','UDROFF',
    #                 #     'Qs', 'SouthQin', 'SouthQout', 'Qn', 'NorthQin', 'NorthQout', 
    #                 #     'Qw', 'WestQin', 'WestQout', 'Qe','EastQin', 'EastQout',
    #                 #     'ZQ', 'MQ', 
    #         ]

    # print(Hdf_total_event.columns)
    # print(Wdf_total_event.columns)
    DiffDf_event = Hdf_total_event[NumericalColumns] - Wdf_total_event[NumericalColumns]
    return Wdf_total_event[NumericalColumns], Hdf_total_event[NumericalColumns], DiffDf_event[NumericalColumns]
Wdf_total_event_Positive, Hdf_total_event_Positive, DiffDf_event_Positive = extract_event_data(Wdf_total, Hdf_total, FeedbackState= 'Positive')
Wdf_total_event_Negative, Hdf_total_event_Negative, DiffDf_event_Negative = extract_event_data(Wdf_total, Hdf_total, FeedbackState= 'Negative')
# =============================================================================
Wdf_total_event_Positive_mean = Wdf_total_event_Positive.mean(numeric_only=True).round(2)
Wdf_total_event_Negative_mean = Wdf_total_event_Negative.mean(numeric_only=True).round(2)
DiffDf_event_Positive_mean = DiffDf_event_Positive.mean(numeric_only=True).round(2)
DiffDf_event_Negative_mean = DiffDf_event_Negative.mean(numeric_only=True).round(2)
# 按列求平均并保存（保留两位小数）
(Wdf_total_event_Positive_mean).to_csv('Wdf_total_event_Positive.csv', header=True)
(Wdf_total_event_Negative_mean).to_csv('Wdf_total_event_Negative.csv', header=True)
(DiffDf_event_Positive_mean).to_csv('DiffDf_event_Positive_mean.csv', header=True)
(DiffDf_event_Negative_mean).to_csv('DiffDf_event_Negative_mean.csv', header=True)

In [ ]:
# 合并四个Series为一个DataFrame并保存
result_df = pd.concat([
    Wdf_total_event_Positive_mean.rename('Positive'),
    DiffDf_event_Positive_mean.rename('Diff_Positive'),
    Wdf_total_event_Negative_mean.rename('Negative'),
    DiffDf_event_Negative_mean.rename('Diff_Negative')
], axis=1)

result_df.to_csv('Event_Mean_Compare.csv', header=True)

In [ ]:
def DrawBar(ax, ColumnIndex, cols, subTitle, dataframe, BarYrange, IF_Xticks=True, IF_Yticks=True):
    # 计算每列的中位数，并将其填充到 NaN 值中
    # df.fillna(df.median(), inplace=True)
    fontsize = 8.5
    bar_labels = list(cols.keys())
    MEANdf = dataframe[bar_labels]
    MEANdf = MEANdf.mean(numeric_only=True)
    counts = MEANdf.tolist()
    # 选择需要绘制的变量
    color_list = [sub_dict['color'] for sub_dict in cols.values()]
    bars = ax.bar(bar_labels, counts, label=bar_labels, color=color_list, edgecolor=color_list, alpha=0.6, linewidth=1)
    bars = ax.bar(bar_labels, counts, label=bar_labels, color='none', edgecolor=color_list, alpha=1, linewidth=1)
    # 添加数据标签
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height+0.005),
                    xytext=(0, 0),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom',
                    fontsize=fontsize-1,
                    style='italic',
                    
                    bbox=dict(boxstyle='round,pad=0.2',  # 背景框样式：圆角，内边距 0.5
          fc='white',  # 背景颜色：黄色
          alpha=0.5,
          ec='none') # 背景透明度：0.5
                    )
    add_TickGrid(ax, IFminorOn=True)
    # ax.set_title(subTitle,fontsize=fontsize)
    ax.set_ylim(BarYrange)
    # yticks = ax.get_yticks()  # 获取当前刻度的位置
    # ax.set_yticks(yticks)     # 固定刻度位置
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=fontsize)
    ax.axhline(0, color="#FF242488", linestyle='--', linewidth=0.8)  # 添加零刻度线
    if IF_Xticks:
        xticks = ax.get_xticks()  # 获取当前刻度的位置
        ax.set_xticks(xticks)     # 固定刻度位置
        # ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='center', fontsize=fontsize) 
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=fontsize) # 旋转 45 度，右对齐
    else:
        # ax.set_xticks([])
        ax.set_xticklabels([])
    if IF_Yticks:
        yticks = ax.get_yticks()  # 获取当前刻度的位置
        ax.set_yticks(yticks)     # 固定刻度位置
        ax.set_yticklabels(ax.get_yticklabels(), fontsize=fontsize) 
    else:
        ax.set_yticks([])
        ax.set_yticklabels([])
    ax.text(0.02, 0.95, f"({get_letter(ColumnIndex)}) {subTitle}", transform=ax.transAxes, 
        fontsize=fontsize+0.5, 
        # fontweight='bold',
        va='top', ha='left')
    pass


AWBcols3 = {
    'Prec': {'color': '#1C1C1C', 'linestyle': '-'},
    'HADV': {'color': "#1E53FF", 'linestyle': '-'},
    'VADV': {'color': '#FF7F24', 'linestyle': '-'},
    'Qs': {'color': "#FF5724", 'linestyle': '-'},
    'Qn': {'color': "#24CCFF", 'linestyle': '--'},
    'Qe': {'color': "#FFA51E", 'linestyle': '-'},
    'Qw': {'color': '#1E90FF', 'linestyle': '--'},
    'PW': {'color': '#7D26CD', 'linestyle': '-'},
    'ET': {'color': "#518B22", 'linestyle': '-'},
}
subTitle = None
IF_legend = False
IF_Diff = True
PlotYrange = (-4, 4)

Xinterval = 24*10#Hour
IF_Xticks=True
plt.rcParams['axes.edgecolor'] = 'black'  # 边框颜色
plt.rcParams['axes.linewidth'] = 0.5     # 边框线宽
LenVar = len(AWBcols3)
# -----------------------------------------------------
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(9./9*LenVar/1.1, 2*3/1.1), dpi=350,
                        # sharex = True,
                        # sharey=True,
                        )#, sharey='row') gridspec_kw={'width_ratios': [5, 5, 5, 4]}
# 修改部分1：在开始绘图前设置全局rcParam参数
BarYrange = (-0.4, 0.8)
BarYrange = (-200,200)
DrawBar(axs[0,0], 0, AWBcols3, 'Positive WRF-S', Wdf_total_event_Positive, BarYrange, IF_Xticks= False)
BarYrange = (-200,200)
DrawBar(axs[0,1], 1, AWBcols3, 'Negative WRF-S', Wdf_total_event_Negative, BarYrange, IF_Xticks= False, IF_Yticks=False)
# axs[0,0].set_ylabel('(mm/day)', fontsize=10)
DrawBar(axs[1,0], 0, AWBcols3, 'Positive WRF-H', Hdf_total_event_Positive, BarYrange, IF_Xticks= False)
BarYrange = (-200,200)
DrawBar(axs[1,1], 1, AWBcols3, 'Negative WRF-H', Hdf_total_event_Negative, BarYrange, IF_Xticks= False, IF_Yticks=False)
axs[1,0].set_ylabel('(mm/day)', fontsize=10)
BarYrange = (-5,15)
DrawBar(axs[2,0], 2, AWBcols3, 'Positive Diff', DiffDf_event_Positive, BarYrange, IF_Xticks= IF_Xticks)
BarYrange = (-0.8, 0.4)
BarYrange = (-15,5)
DrawBar(axs[2,1], 3, AWBcols3, 'Negative Diff', DiffDf_event_Negative, BarYrange, IF_Xticks= IF_Xticks, IF_Yticks=False)
# axs[2,0].set_ylabel('(mm/day)', fontsize=10)
plt.subplots_adjust(wspace=0.05, hspace=0.1)